In [1]:
import os
import glob
import base64

# importing and looking at .dxf file
import ezdxf # pip install ezdxf https://pypi.org/project/ezdxf/

from matplotlib.path import Path
import numpy as np
import pandas as pd
import csv


# Importing stuff for plotly to standardise figures
Not sure why the default background is blue...

In [3]:
import plotly
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"


default_layout = dict(template="plotly_white",
                      xaxis=dict(
                          mirror=True,
                          ticks='outside',
                          showline=True,
                          linecolor='black',
                      ),
                      yaxis=dict(
                          mirror=True,
                          ticks='outside',
                          showline=True,
                          linecolor='black',
                      ),
                      )


def apply_default_layout(fig):
    fig.update_layout(**default_layout)
    fig.update_xaxes(default_layout['xaxis'])
    fig.update_yaxes(default_layout['yaxis'])
    return fig


def default_fig():
    fig = go.Figure()
    apply_default_layout(fig)
    return fig


def update_plotly_layout(fig):
    fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        family="Serif",
        size=18,
        color="Black"
    )
)

In [4]:
cwd = os.getcwd()
print(cwd)

dxf_file = glob.glob(f'{cwd}/*.dxf')
print(dxf_file)

/Users/johanndrayne/Documents/UBC_Course/ELEC_542/Project/code
['/Users/johanndrayne/Documents/UBC_Course/ELEC_542/Project/code/KondoEntropy_double_square.dxf']


# Turning .dxf to POLYLINE. 

In [5]:
doc = ezdxf.readfile(dxf_file[0])
msp = doc.modelspace()

polyline_points = {}
# entity query for all POLYLINE entities in modelspace
# https://ezdxf.readthedocs.io/en/stable/dxfentities/polyline.html
for index, e in enumerate(msp.query("POLYLINE")):
    val = f'val_{index}'
    points = np.array(list(e.points()))
    polyline_points[val] = np.vstack((points, points[0]))

    

In [6]:
initial_fig = default_fig()

for key, val in polyline_points.items():
    # print(key, val)
    x_data = val[:,0]
    y_data = val[:,1]

    initial_fig.add_trace(go.Scatter(x=x_data, y=y_data,
                        mode='lines',
                        marker_line_width=3,
                        name=f'{key}',
                        # fill="toself"
                        ))


initial_fig.update_layout(title=f'Plotting {dxf_file}',
                  yaxis_zeroline=True,
                  xaxis_zeroline=True,
#                   width = 800,
#                   height = 800
)

update_plotly_layout(initial_fig)
initial_fig.update_yaxes(scaleanchor = "x", scaleratio = 1,)

initial_fig.show()

# Importing stuff for plotly to standardise figures
Not sure why the default background is blue...

In [7]:
# seting x and y range to calculate potential in
x_range = np.array([9.2, 10.6])
y_range = np.array([1.2, 1.4])
# number of points in x and y axis to discretise design
nx = 200
ny = 15

max_range = np.max([np.max(y_range) - np.min(y_range), np.max(x_range) - np.min(x_range)])

plot_info = {
    'x_range':x_range,
    'y_range':y_range,
    'max_range':max_range,
    'x_range_to_plot':np.array([-max_range, max_range])/2 + np.mean(x_range),
    'y_range_to_plot':np.array([-max_range, max_range])/2 + np.mean(y_range),
    'nx':nx,
    'ny':ny,
}




discretised_gates = {}

x_axis = np.linspace(np.min(x_range), np.max(x_range), nx)
y_axis = np.linspace(np.min(y_range), np.max(y_range), ny)
z_data = np.zeros((ny, nx))

xx_axis, yy_axis = np.meshgrid(x_axis, y_axis)
coors=np.hstack((xx_axis.reshape(-1, 1), yy_axis.reshape(-1,1))) # coors.shape is (4000000,2)

fig = default_fig()


for key, val in polyline_points.items():
    poly_path=Path(val[:,0:2])

    mask = poly_path.contains_points(coors)
#     print(f'mask {np.shape(mask)} :: nx {nx}')
    mask_2d = np.reshape(mask, (-1, nx))


    x_data = xx_axis[mask_2d]
    y_data = yy_axis[mask_2d]

    if np.sum(mask_2d.astype(int)) > 0:
        gate_dict = {
            'coordinates': mask_2d.astype(int),
            'gate_val': 1,
        }
        discretised_gates[key] = gate_dict
        z_data = z_data + mask_2d.astype(int)

discretised_gates['x_axis'] = x_axis
discretised_gates['y_axis'] = y_axis


fig.add_trace(go.Heatmap(
    z=z_data,
    x=x_axis,
    y=y_axis,
    colorscale='Greens',
    opacity=1))

    # print(mask_2d.astype(int))


fig.update_layout(title=f'Plotting {dxf_file}',
                  yaxis_zeroline=True,
                  xaxis_zeroline=True,
                  width = 800,
                  height = 800,
                  xaxis_range = plot_info['x_range_to_plot'],
                  yaxis_range = plot_info['y_range_to_plot'],
)

update_plotly_layout(fig)
fig.update_yaxes(scaleanchor = "x", scaleratio = 1,)

fig.show()
# plt.imshow(mask.reshape(height, width))


# Re-plotting from discretised_gates

In [8]:
def plot_discretised_gates(discretised_gates, plot_info, 
                           plot_type='coordinates', plot=True, colorscale='Greens',
                          color_range=None):
    
    x_axis = discretised_gates['x_axis']
    y_axis = discretised_gates['y_axis']
    

    fig = default_fig()

    z_data = np.zeros((np.shape(y_axis)[0], np.shape(x_axis)[0]))

    for key, val in discretised_gates.items():
        if 'val_' in key:

            z_data += val[plot_type]*val['gate_val']
            
    if color_range is None:
        zmin, zmax = np.min(z_data), np.max(z_data)
    else: 
        zmin, zmax = color_range[0], color_range[1]
        
    fig.add_trace(go.Heatmap(
        z=z_data,
        x=x_axis,
        y=y_axis,
        colorscale=colorscale,
        zmin = zmin,
        zmax = zmax,
        opacity=1))
    
    
            
    fig.update_layout(
#         title=f'Plotting {dxf_file}',
                  yaxis_zeroline=True,
                  xaxis_zeroline=True,
#                   width = 800,
#                   height = 800,
#                   xaxis_range = plot_info['x_range_to_plot'],
#                   yaxis_range = plot_info['y_range_to_plot'],
                     )
                     
    update_plotly_layout(fig)
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1,)

    
    if plot:
        fig.show()

    return fig


# plot_discretised_gates(discretised_gates, plot_info)
# print('')

# Notes on calulating the 2deg potential 

## rectangular gates
We want to calculate the potential 
$$
\phi(R) = \frac{1}{\pi}V_g\arctan\frac{1}{R-x-y}
$$

With some manipulation:
$$
\phi(R) = \frac{1}{2\pi}V_g
\left(
\frac{\pi}{2} + 
\arctan\frac{x}{d} + 
\arctan\frac{y}{d} + 
\arctan\frac{xy}{dR}
\right)
$$

## polygon gates
We want to calculate the potential 
$$
\phi(R) = \frac{1}{2\pi}\sum_iV_iI_i(R)
$$

Where:
- $V_i$ is the potential on the gate.
- $i$ represents each gate (i.e. if we have two gates i=2)

$$
I_i(R) = \sum_m
\left[
\arctan(\sin\gamma_m\cot\alpha_m)
+
\arctan(\sin\gamma_m\cot\beta_m)
\right]
+2\pi C_i(r)
$$

Where:
- $\frac{l_1}{H} = \cot\beta$ 
- $\frac{l_2}{H} = \cot\alpha$ 
- $\frac{z}{R} = \sin\gamma$ 
- $m$ represents each discretised point within the gate
- $C_i = 1$ if underneath the gate

In [9]:
def get_potential_from_gate(discretised_gates, material_info):
    x_axis = discretised_gates['x_axis']
    y_axis = discretised_gates['y_axis']
    del_x_half = np.abs((x_axis[1] - x_axis[0])/2)
    del_y_half = np.abs((y_axis[1] - y_axis[0])/2)

    xx_axis, yy_axis = np.meshgrid(x_axis, y_axis)

    len_x = np.shape(x_axis)[0]
    len_y = np.shape(y_axis)[0]

    d = material_info['2deg_depth']

    def get_g(u, v):
        R = np.sqrt(u**2 + v**2 + d**2)
        return np.arctan((u*v)/(d*R))/(2*np.pi)


    for key, val in discretised_gates.items():
        if 'val_' in key:
            potential_data = np.zeros((len_y, len_x))

            gate_coords = val['coordinates']

            for x_index, x in enumerate(x_axis):
                for y_index, y in enumerate(y_axis):

                    if gate_coords[y_index, x_index] != 0:
                        L = x - del_x_half
                        R = x + del_x_half
                        B = y - del_y_half
                        T = y + del_y_half

                        g1 = get_g(xx_axis-L, yy_axis-B)
                        g2 = get_g(xx_axis-L, -yy_axis+T)
                        g3 = get_g(-xx_axis+R, yy_axis-B)
                        g4 = get_g(-xx_axis+R, -yy_axis+T)
                        potential_data += g1 + g2 + g3 + g4

            discretised_gates[key]['potential'] = potential_data*val['gate_val']

    return discretised_gates


def save_geometric_potential_to_csv(discretised_gates, csv_name = 'geometric_potential.csv'):
    csv_df = pd.DataFrame([])
    
    gate_num = 0
    for key, val in discretised_gates.items():
        if 'val_' in key:
                temp_df = pd.DataFrame({f'val_{gate_num}_coordinates':discretised_gates[key]['coordinates'].flatten(),
                                       f'val_{gate_num}_potential':discretised_gates[key]['potential'].flatten()})
                csv_df = pd.concat([csv_df, temp_df], axis=1) 
                gate_num += 1 
        else:
            temp_df = pd.DataFrame({key:discretised_gates[key].flatten()})
            csv_df = pd.concat([csv_df, temp_df], axis=1) 

    csv_df.to_csv(f'{csv_name}', index=False)


def get_1d_to_2d(array, nx, ny):
    return np.array(array).reshape(ny, nx)



In [10]:
def get_discretised_gates_from_csv(nx, ny, csv_name = 'geometric_potential.csv'):
    
    df = pd.read_csv(csv_name)
    column_names = df.columns
    
    discretised_gates = {}
    gate = {}
    
    old_val = ''
    for index, column_name in enumerate(df):
        val = ''.join(column_name.split('_')[0:2])
        
        if 'val' in val:
            gate[column_name.split('_')[-1]] = get_1d_to_2d(df[column_name], nx, ny)
            try:
                if ''.join(column_names[index+1].split('_')[0:2]) != val:
                    discretised_gates['_'.join(column_name.split('_')[0:2])] = gate
                    gate = {}
            except:
                continue
        else:
            discretised_gates[column_name] = df[column_name].dropna().to_numpy()
            
    return discretised_gates

In [11]:
# get_discretised_gates_from_csv(initial_nx, initial_ny, csv_name = 'geometric_potential.csv')



# Creating Dash functions to interact with potential 

## Initial plotting functions

In [12]:
from dash import Dash, dcc, html, Input, Output, State

In [13]:
def create_n_sliders(num_sliders, app_layout):
    slider_layouts = []
    app_inputs = []
     
    for i in range(num_sliders):   
        key = f'val_{i}'

        slider_layout = [
            html.P(f'{key}'),
            dcc.RangeSlider(
                id=f'{key}-potential-slider',
                min=0, max=1, step=0.01,
                marks={i: '{}'.format(i) for i in np.linspace(0,1,5)},
                value=[1.0],
                tooltip={"placement": "bottom", "always_visible": True})
                ]

        slider_layouts = slider_layouts + slider_layout

        app_input = [Input(f'{key}-potential-slider', 'value')]
        app_inputs = app_inputs + app_input

    slider_layouts = html.Div(slider_layouts, 
                              style={'width': '39%', 'float': 'right', 'display': 'inline-block'})
    
    app_layout.append(slider_layouts)

    return app_layout, app_inputs


def get_discretised_gates(plot_info, polyline_gates):
    discretised_gates = {}
    nx = plot_info['nx']
    ny = plot_info['ny']

    x_axis = np.linspace(np.min(plot_info['x_range']), 
                         np.max(plot_info['x_range']), 
                         nx)
    y_axis = np.linspace(np.min(plot_info['y_range']), 
                         np.max(plot_info['y_range']), 
                         ny)
    z_data = np.zeros((ny, nx))

    xx_axis, yy_axis = np.meshgrid(x_axis, y_axis)
    coors=np.hstack((xx_axis.reshape(-1, 1), yy_axis.reshape(-1,1))) # coors.shape is (4000000,2)

    gate_num = 0
    for key, val in polyline_gates.items():
        
        x_data = val['x_array']
        y_data = val['y_array']
        
        poly_path=Path(np.stack((x_data, y_data), axis=1))
        mask = poly_path.contains_points(coors)
        mask_2d = np.reshape(mask, (-1, nx))


        x_data = xx_axis[mask_2d]
        y_data = yy_axis[mask_2d]

        if np.sum(mask_2d.astype(int)) > 0:
            gate_dict = {
                'coordinates': mask_2d.astype(int),
                'gate_val': 1,
            }
            discretised_gates[f'val_{gate_num}'] = gate_dict
            gate_num += 1
            z_data = z_data + mask_2d.astype(int)


    discretised_gates['x_axis'] = x_axis
    discretised_gates['y_axis'] = y_axis

    return discretised_gates


def get_plot_info(depth_2deg, minx, maxx, miny, maxy, nx, ny):
    
    max_range = np.max([maxy - miny, maxx - minx])
    mean_x = (minx + maxx)/2
    mean_y = (miny + maxy)/2

    plot_info = {
        'x_range':[minx, maxx],
        'y_range':[miny, maxy],
        'max_range':max_range,
        'x_range_to_plot':np.array([-max_range, max_range])/2 + mean_x,
        'y_range_to_plot':np.array([-max_range, max_range])/2 + mean_y,
        'nx':nx,
        'ny':ny,
        '2deg_depth':depth_2deg
    }
    
    return plot_info


def save_file(name, content, upload_directory):
    """Decode and store a file uploaded with Plotly Dash."""
    if name is not None:
        data = content.encode("utf8").split(b";base64,")[1]
        with open(os.path.join(upload_directory, name), "wb") as fp:
            fp.write(base64.decodebytes(data))
        
def save_dxf_to_csv(name, upload_directory):
    file_path = f'{upload_directory}/{name}'
    doc = ezdxf.readfile(file_path)
    msp = doc.modelspace()

    polyline_points = {}
    # entity query for all POLYLINE entities in modelspace
    # https://ezdxf.readthedocs.io/en/stable/dxfentities/polyline.html
    for index, e in enumerate(msp.query("POLYLINE")):
        val = f'val_{index}'
        points = np.array(list(e.points()))
        polyline_points[val] = np.vstack((points, points[0]))
        
    csv_df = pd.DataFrame([])
        
    for key, val in polyline_points.items():
        temp_df = pd.DataFrame({f'{key}_xcoord':val[:,0],
                               f'{key}_ycoord':val[:,1]})
        csv_df = pd.concat([csv_df, temp_df], axis=1)
        
    csv_df.to_csv(f"{upload_directory}/{name.split('.')[0]}.csv", index=False)

    
def read_csv_to_polyline(name, upload_directory):
    csv_name = f"{upload_directory}/{name.split('.')[0]}.csv"
    df = pd.read_csv(csv_name)
    column_names = df.columns    
    polyline_gates = {}

    for index in range(int(len(column_names)/2 - 1)):
        val = ''.join(column_names[int(2*index)].split('_')[0:2])
        
        x_array = df[column_names[int(2*index)]].dropna().to_numpy()
        y_array = df[column_names[int(2*index+1)]].dropna().to_numpy()
        
        gate = {'x_array':x_array,
                'y_array':y_array,
               }
        
        polyline_gates[val] = gate
        gate = {}
        
    return polyline_gates
        

def plot_polyline(name, upload_directory):
    polyline_gates = read_csv_to_polyline(name, upload_directory)
    
    fig = default_fig()
        
    for key, val in polyline_gates.items():
        x_data = val['x_array']
        y_data = val['y_array']

        fig.add_trace(go.Scatter(x=x_data, y=y_data,
                            mode='lines',
                            marker_line_width=3,
                            name=f'{key}',
                            # fill="toself"
                            ))


    fig.update_layout(title=f'Plotting {dxf_file}',
                      yaxis_zeroline=True,
                      xaxis_zeroline=True,
    )

    update_plotly_layout(fig)
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1,)

    return fig


def uploaded_files(upload_directory):
    """List the files in the upload directory."""
    files = []
    for filename in os.listdir(upload_directory):
        path = os.path.join(upload_directory, filename)
        if os.path.isfile(path):
            files.append(filename)
    return files



In [ ]:
# https://docs.faculty.ai/user-guide/apps/examples/dash_file_upload_download.html
# INITIAL SETTING UP
initial_fig = default_fig()
num_sliders = 20
cwd = os.getcwd()
UPLOAD_DIRECTORY = f"{cwd}/uploaded_files"
fig_style={ 'verticalAlign': 'middle', 'width': '70vh', 'height': '70vh'}

if not os.path.exists(UPLOAD_DIRECTORY):
    os.makedirs(UPLOAD_DIRECTORY)

# Dash Layouts
setup_layout =  [
    html.Div([
        html.H1('2DEG yodel'),
        html.H2('Upload .dxf'),
        dcc.Upload(id='upload-data',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select a File')
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            }),
         html.H3('Set the Inputs'),
         dcc.Graph(id='initial-gate-graph', figure=initial_fig, style = fig_style),
         html.H5('Depth of 2DEG'),
         dcc.Input(id="2deg-depth",type='number', placeholder="2deg_depth in um", value=0.09),
         html.H5('Zoom to [xmin, xmax] and [ymin, ymax]'),
         dcc.Input(id="min-x-potential",type='number', placeholder="min x range", value=9.6),
         dcc.Input(id="max-x-potential",type='number', placeholder="max x range", value=10.7),
         dcc.Input(id="min-y-potential",type='number', placeholder="min y range", value=0.5),
         dcc.Input(id="max-y-potential",type='number', placeholder="max y range", value=1.5),
         html.H5('Number of points in x direction and y direction'),
         dcc.Input(id="numpts-x-potential",type='number', placeholder="numpts x",  value=50),
         dcc.Input(id="numpts-y-potential",type='number', placeholder="numpts y",  value=50),
         html.Button('Update Gates', id='update-gate'),
         dcc.Graph(id='discretised-gate-graph', figure=initial_fig, style = fig_style),
         html.Button('Update Potential', id='update-potential'),
        ]), 
    html.Div(id='dummy1'),
    html.Div(id='dummy2')]


potential_layout =  [
    html.Div(
        [html.H4('Potential chane in 2DEG'),
         dcc.Graph(id='potential-graph'),], style={'width': '59%', 'height': '59%', 'display': 'inline-block'},
    )
]
    
potential_layout, app_inputs = create_n_sliders(num_sliders, potential_layout)
app_layout = setup_layout + potential_layout

app = Dash(__name__)

app.layout = html.Div(app_layout)


# # uploading .dxf
@app.callback(
    Output('initial-gate-graph', 'figure'),
    Input("upload-data", "filename"),
    Input("upload-data", "contents"),
)
def update_output(filename, uploaded_file_content):
    """Save uploaded files and regenerate the file list."""
    if filename is not None:
        save_file(filename, uploaded_file_content, UPLOAD_DIRECTORY)
#         files = uploaded_files(UPLOAD_DIRECTORY)
#         if len(files) == 0:
#             return [html.Li("No files yet!")]
#         else:
        save_dxf_to_csv(filename, UPLOAD_DIRECTORY)
        return plot_polyline(filename, UPLOAD_DIRECTORY) 
    else:
        return default_fig()
    
    

# updating discretisation
@app.callback(
    Output('discretised-gate-graph', 'figure'),
    Input('update-gate', 'n_clicks'),
    State('2deg-depth', 'value'),
    State('min-x-potential', 'value'),
    State('max-x-potential', 'value'),
    State('min-y-potential', 'value'),
    State('max-y-potential', 'value'),
    State('numpts-x-potential', 'value'),
    State('numpts-y-potential', 'value'),
    State("upload-data", "filename")
    )
def update_discretised_gates(update_gate, depth_2deg, minx, maxx, miny, maxy, nx, ny, filename):
    if filename is not None:
        polyline_gates = read_csv_to_polyline(filename, UPLOAD_DIRECTORY)

        plot_info = get_plot_info(depth_2deg, minx, maxx, miny, maxy, nx, ny)    
        discretised_gates = get_discretised_gates(plot_info, polyline_gates)
        discretised_fig = plot_discretised_gates(discretised_gates, plot_info, plot=False,)

        return discretised_fig
    else:
        return default_fig()



# updating discretisation
@app.callback(
    Output("dummy2", "children"),
    Input('update-potential', 'n_clicks'),
    State('2deg-depth', 'value'),
    State('min-x-potential', 'value'),
    State('max-x-potential', 'value'),
    State('min-y-potential', 'value'),
    State('max-y-potential', 'value'),
    State('numpts-x-potential', 'value'),
    State('numpts-y-potential', 'value'),
    State("upload-data", "filename")
    )
def update_potential_csv(update_potential_csv, depth_2deg, minx, maxx, miny, maxy, nx, ny, filename):
    if filename is not None:
        polyline_gates = read_csv_to_polyline(filename, UPLOAD_DIRECTORY)

        plot_info = get_plot_info(depth_2deg, minx, maxx, miny, maxy, nx, ny)
        discretised_gates = get_discretised_gates(plot_info, polyline_gates)
        material_info = {'2deg_depth': depth_2deg}
        discretised_gates_new = get_potential_from_gate(discretised_gates, material_info)
        save_geometric_potential_to_csv(discretised_gates_new, csv_name = 'geometric_potential.csv')
        return None


# updating potential
@app.callback(
    Output('potential-graph', 'figure'),
    State("upload-data", "filename"),
    State('numpts-x-potential', 'value'),
    State('numpts-y-potential', 'value'),
    app_inputs    
    )
def update_potential(filename, nx, ny, *slider_vals):
    if filename is not None:
        discretised_gates = get_discretised_gates_from_csv(nx, ny, csv_name = 'geometric_potential.csv')
        z_data = 0
        index = -1
        for key, val in discretised_gates.items():
            if 'val_' in key:
                index += 1
                discretised_gates[key]['gate_val'] = slider_vals[index][0]
                z_data = z_data + discretised_gates[key]['potential']


        color_range = [np.min(z_data), np.max(z_data)]
        potential_fig = plot_discretised_gates(discretised_gates, plot_info,  
                                               plot_type='potential', plot=False, colorscale='Plotly3', color_range=color_range) 

        return potential_fig
    else:
        return default_fig()


if __name__ == "__main__":
    app.run_server(port=4444, debug=False
    )

Dash is running on http://127.0.0.1:4444/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4444
Press CTRL+C to quit
127.0.0.1 - - [27/Mar/2023 22:09:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:26] "GET /_dash-component-suites/dash/dcc/async-upload.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:09:27] "POST /_dash-updat

---
---
---